# Модули, настройки, функции

In [ ]:
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from pandas import Series

from datetime import datetime, timedelta
import itertools
import ast
from itertools import combinations
from scipy.stats import ttest_ind, pearsonr

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler

from wordcloud import WordCloud, STOPWORDS 
import string
import re
import nltk
from nltk.util import ngrams
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures

import json

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingRegressor
import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings("ignore")

Функции:

In [ ]:
# Вычисление метрики
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))


# Преобразование в None для строковых признаков
def set_None(value):
    if value is None:
        return value
    else:        
        str_value = str(value).strip()
        if str_value == '' or str_value.lower() == 'nan' or str_value.lower() == 'none':
            return None
        else:
            return str_value


# перевод в нижний регистр
def set_lower(value):
    if value is None:
        return value
    else:
        return value.lower()
    
    
# Определение выбросов
def get_outlier(df, col):
    Q3 = pd.DataFrame.quantile(df, q=0.75, axis=0, numeric_only=True, interpolation='midpoint')[col]
    Q1 = pd.DataFrame.quantile(df, q=0.25, axis=0, numeric_only=True, interpolation='midpoint')[col]
    IQR = round(Q3-Q1,1)
    return df[~df[col].between(Q1 - 1.5*IQR, Q3 + 1.5*IQR)][col], Q1 - 1.5*IQR, Q3 + 1.5*IQR


# Информация о выбросах с графиками
def show_info(df, col, show=True):
    # Выводим количество выбросов и их границы
    out, lim1, lim2 = get_outlier(df, col)
    minCol = df[col].min()
    maxCol = df[col].max()
    median = df[col].median()
    nulCol = sum(pd.isnull(df[col]))
    
    cnt = min(int(df[col].value_counts().count()),2000)
    
    if show:
        print('Не заполнено: ', nulCol)
        print('Минимум: ', minCol)
        print('Максимум: ', maxCol)
        print('Медиана: ', median)
        print('Количество выбросов: ', len(out))
        if len(out) > 0:
            print('Нижняя граница выбросов: ', lim1)
            print('Верхняя граница выбросов: ', lim2)

        # Выводим графики: гистограмму и боксплот
        fig, axes = plt.subplots(1,2,figsize=(12,4))
        axes[0].hist(df[col], bins=cnt)
        axes[1].boxplot(df[col])
    
    return {'med': median, 'lm1': lim1, 'lm2': lim2}


# График с боксплотами
def show_boxplot(df, x_col, y_col):
    plt.figure(figsize=(12, 8))
    g = sns.boxplot(y=y_col, x=x_col, data=df, color='yellow')
    g.set_title(y_col + ' of ' + x_col, fontsize=20)
    g.set_ylabel(y_col, fontsize=15)
    g.set_xticklabels(g.get_xticklabels(),rotation=45)
    plt.show()
    
# Сравнение двух графиков 
def val_log_plot(df, col):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    ax[0].hist(df[col], rwidth=0.9, alpha=0.7, bins=15)
    ax[0].set_title(col)

    ax[1].hist(np.log(df[col]+1), rwidth=0.9, alpha=0.7, bins=15)
    ax[1].set_title('log of '+col)

    plt.show()
    

            
# функция вычисления попарного p_value при множественном значении категориального столбца
def get_stat_dif(df, columns, col_target):
    
    p_list =[]
    res = '' 
    for col in columns:
        cols = df.loc[:, col].value_counts().index
        combinations_all = list(combinations(cols, 2))
        if len(combinations_all)==0:
            res = col + ': значимости нет'
            print(res)
            continue
        
        conv = 0.05 / len(combinations_all) # пороговый уровень значимости с поправкой Бонферрони
    
        for comb in combinations_all:
            p_value = ttest_ind(df.loc[df.loc[:, col] == comb[0], col_target], 
                     df.loc[df.loc[:, col] == comb[1], col_target]).pvalue
            p_list.append(p_value)
        
            if p_value < conv:
                res = ', значимость есть: для ' + str(comb) + ' p_value=' + str(p_value)
            else:
                res = ', значимости нет'
            break
            
        if res == '':
            res = ', значимости нет: min(p_value)=' + str(min(p_list))
    
        res = col + ': порог=' + str(round(conv,6)) + res
        print(res)
        
        
def get_stat_corr(df, columns, col_target):
    
    for col in columns:
        p_value = pearsonr(df[col], df[col_target])[1]
    
        if (p_value / len(columns)) < 0.05:
            print(col + ' для ' + col_target + ' - значимость есть')
        else:
            print(col + ' для ' + col_target + ' - значимости нет')
        

In [ ]:
RANDOM_SEED = 42
VAL_SAZE = 0.15

# Чтение данных

In [ ]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [ ]:
train.info()

In [ ]:
train.nunique()

In [ ]:
test.info()

In [ ]:
test.nunique()

In [ ]:
# Для быстрой обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 
test['sample'] = 0 
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

df = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, df.shape)

# EDA

Каждый признак рассмотрим и обработаем отдельно.

### price

In [ ]:
df[df['sample']==1].price.min(), df[df['sample']==1].price.max()

In [ ]:
# Наглядное влияние логарифма
val_log_plot(df[df['sample']==1], 'price')

In [ ]:
# Введем еще одну переменную - логарифм цены для проверки на ней значимости признаков
df['price_log'] = np.log(df['price']+1)

### model_info

In [ ]:
display(train['model_info'].unique()[:10])
display(test['model_info'].unique()[:10])

In [ ]:
df.model_info.unique(), len(df.model_info.unique())

In [ ]:
df.model_info = df.model_info.apply(set_None)

In [ ]:
df[df.model_info.isnull()].shape

In [ ]:
# Строка с отстутсвующей моделью всего одна. Заполним модель по названию автомобиля.
nm = df[df.model_info.isnull()].iloc[0]['name']
m_i = df[df['name']==nm].iloc[0].model_info
df.model_info = df.model_info.fillna(m_i)

### bodyType

In [ ]:
display(df['bodyType'].value_counts())

In [ ]:
df['bodyType'].unique()

In [ ]:
# Выделим крупные группы кузовов. Для этого возьмем первые слова в их названиях

def get_body(value):
    if value is None:
        return value
    else:
        main_value = str(value).strip().split()[0].split('-')[0]
        return main_value

    
df['body'] = df['bodyType'].apply(get_body)
df['body'].unique()

### brand

In [ ]:
print(df['brand'].unique())

### color

In [ ]:
print(df['color'].unique())

In [ ]:
df['color'].value_counts()

In [ ]:
# Сделаем признак популярности цвета
# color_rare = ['жёлтый', 'оранжевый', 'пурпурный', 'фиолетовый', 'розовый']
color_freq = ['чёрный', 'белый',  'серый', 'синий']
# df['color_type'] = df['color'].apply(lambda x: 0 if x in color_freq else 2 if x in color_rare else 1)
df['color_type'] = df['color'].apply(lambda x: 0 if x in color_freq else 1)
df['color_type'].value_counts()

Посмотрим на влияние цвета на цену

In [ ]:
show_boxplot(df[df['sample']==1], 'color', 'price_log')

In [ ]:
show_boxplot(df[df['sample']==1], 'color_type', 'price_log')

### enginePower

In [ ]:
df.enginePower.unique()

In [ ]:
df['enginePower'] = df['enginePower'].apply(lambda x: str(x).split()[0])
df['enginePower'] = df['enginePower'].apply(lambda x: int(x))

In [ ]:
# Добавим столбец с налоговой ставкой на мощность автомобиля
bins = [0, 100, 125, 150, 175, 200, 225, 250, 801]
labels = ['12','25','35', '45', '50', '65', '75', '150']
df['tax_rate'] = pd.cut(df['enginePower'], bins=bins, labels=labels)
df['tax_rate'] = df['tax_rate'].values.astype('int64')

In [ ]:
# Посмотрим на влияние логарифма
val_log_plot(df, 'enginePower')
val_log_plot(df[df['sample']==1], 'enginePower')
val_log_plot(df[df['sample']==0], 'enginePower')

In [ ]:
# Распределение улучшилось. Добавим столбец с логарифмом.
df['eP_log'] = np.log(df['enginePower'] + 1)

In [ ]:
# Проверим наличие выбросов
d = show_info(df[df['sample']==0],'eP_log')

In [ ]:
d = show_info(df[df['sample']==1],'eP_log')

Выбросов мало. Ничего с ними делать не будем, только пометим строки с выбросами.

In [ ]:
# Делаем столбец с пометкой выбросов. Объединяем, так как границы выбросов совпадают.
d = show_info(df,'eP_log', show=False)
df['eP_log_out'] = df['eP_log'].apply(lambda x: 1 if x>d['lm2'] or x<d['lm1'] else 0)

### engineDisplacement

In [ ]:
df.engineDisplacement.unique()

In [ ]:
df[df.engineDisplacement=='undefined LTR'].fuelType.unique()

Все автомобили, у которых неизвестен объем двигателя, - электрические. Заменим у электроавтомобилей объем двигателя на значение от мощности, разделив на 70. Это грубое приближение, но оно лучше медианы от всех машин.

In [ ]:
df[df.engineDisplacement=='undefined LTR'] = df[df.engineDisplacement=='undefined LTR'].apply(lambda x: x.replace('undefined LTR', '0.0 '))
df.engineDisplacement = df.engineDisplacement.apply(lambda x: x[:3])
df.engineDisplacement = df.engineDisplacement.apply(lambda x: float(x))

df.engineDisplacement = df.apply(lambda row: row['engineDisplacement'] if row['engineDisplacement']>0 \
                                          else np.round(row['enginePower']/70, 1), axis=1)

In [ ]:
df.engineDisplacement.unique()

In [ ]:
# Посмотрим на влияние логарифма
val_log_plot(df, 'engineDisplacement')
val_log_plot(df[df['sample']==1], 'engineDisplacement')
val_log_plot(df[df['sample']==0], 'engineDisplacement')

Распределение стало на мизер лучше - пик сместился ближе к центру. Прологарифмируем, так как это дополнительно уменьшит количество выбросов.

In [ ]:
# Логарифмируем
df['eD_log'] = np.log(df['engineDisplacement'] + 1)

In [ ]:
# Выбросы
d = show_info(df[df['sample']==1],'engineDisplacement')

In [ ]:
d = show_info(df[df['sample']==1],'eD_log')

In [ ]:
d = show_info(df[df['sample']==0],'engineDisplacement')

In [ ]:
d = show_info(df[df['sample']==0],'eD_log')

In [ ]:
# Делаем столбец с пометкой выбросов. Объединяем, так как границы выбросов совпадают.
d = show_info(df,'eD_log', show=False)
df['eD_log_out'] = df['eD_log'].apply(lambda x: 1 if x>d['lm2'] or x<d['lm1'] else 0)

### fuelType

In [ ]:
df.fuelType.unique()

### Привод

In [ ]:
df.Привод.unique()

### numberOfDoors

In [ ]:
df.numberOfDoors.unique()

### ПТС

In [ ]:
df.ПТС.unique()

In [ ]:
# Сделаем двоичный признак
pts_dict={'Оригинал': 0, 'Дубликат': 1}
df['ПТС'] = df['ПТС'].map(pts_dict)

### Руль

In [ ]:
df.Руль.unique()

In [ ]:
# Сделаем двоичный признак
rule_dict={'Левый': 0, 'Правый': 1}
df['Руль'] = df['Руль'].map(rule_dict)

### vehicleTransmission

In [ ]:
df.vehicleTransmission.unique()

In [ ]:
# Приведем названия к более коротким
tr_dict={'автоматическая': 'AT','механическая': 'MT','роботизированная': 'RBT', 'вариатор': 'VRT'}
df['vehicleTransmission'] = df['vehicleTransmission'].map(tr_dict)

### vehicleConfiguration

In [ ]:
print(df.vehicleConfiguration[4660:4680])

Столбец содержит данные, которые есть в других столбцах. использовать его не будем.

### mileage

In [ ]:
df[df['sample']==1].mileage.hist(bins=200)

In [ ]:
df[df['sample']==0].mileage.hist(bins=200)

In [ ]:
# Проверим выбросы
d = show_info(df[df['sample']==0],'mileage')

In [ ]:
d = show_info(df[df['sample']==1],'mileage')

In [ ]:
# Делаем столбец с пометкой выбросов. Объединяем, так как границы выбросов практически совпадают.
d = show_info(df,'mileage', show=False)
df['mileage_out'] = df['mileage'].apply(lambda x: 1 if x>d['lm2'] else 0)

In [ ]:
# Посмотрим на влияние логарифма
val_log_plot(df, 'mileage')
val_log_plot(df[df['sample']==1], 'mileage')
val_log_plot(df[df['sample']==0], 'mileage')

Распределение лучше не стало, к тому же опытным путем было выяснено, что логарифмирование значительно увеличивает количество выбросов.    
Вместо логарифмирования просто отнормируем.

In [ ]:
# Статистики у train и test примерно совпадают, поэтому преобразовываем объединенный датасет
m = df['mileage'].mean()
df['mileage_norm'] = df['mileage']/m

In [ ]:
val_log_plot(df, 'mileage_norm')
val_log_plot(df[df['sample']==1], 'mileage_norm')
val_log_plot(df[df['sample']==0], 'mileage_norm')

Распределение логарифма от нормированной величины более похоже на нормальное.

In [ ]:
# Логарифмируем
df['mileage_norm_log'] = np.log(df['mileage_norm'] + 1)

In [ ]:
# Введем признак износа - сколько десятков тысяч километров в год проезжала машина
df['milPerYear'] = df.apply(
    lambda row : 0 if row['productionDate']>2020 else row['mileage']/((2021-row['productionDate'])*10000), axis = 1)

df['milPerYear'].hist(bins=100)

### modelDate

In [ ]:
df.modelDate.unique()

In [ ]:
# Новый столбец с возрастом модели автомобиля, чтобы уменьшить значения
df['model_age'] = df['modelDate'].apply(lambda x: 2021-x)
df['model_age'].hist(bins=60)

In [ ]:
val_log_plot(df, 'model_age')
val_log_plot(df[df['sample']==1], 'model_age')
val_log_plot(df[df['sample']==0], 'model_age')

In [ ]:
# Новый столбец с логарифмом 
df['model_age_log'] = np.log(df['model_age'] + 1)

### productionDate

In [ ]:
df.productionDate.unique()

In [ ]:
# Новый столбец с возрастом автомобиля, чтобы уменьшить значения
df['age'] = df['productionDate'].apply(lambda x: 2021-x)

In [ ]:
# Посмотрим на влияние логарифма
val_log_plot(df, 'age')
val_log_plot(df[df['sample']==1], 'age')
val_log_plot(df[df['sample']==0], 'age')

In [ ]:
# Новый столбец с логарифмом 
df['age_log'] = np.log(df['age'] + 1)

In [ ]:
# Новый столбец с разницей между годом выпуска и годом модели
df['model_old'] = df['productionDate'] - df['modelDate']
print(df['model_old'].unique())

In [ ]:
# Посмотрим на влияние логарифма 
val_log_plot(df, 'model_old')
val_log_plot(df[df['sample']==1], 'model_old')
val_log_plot(df[df['sample']==0], 'model_old')

In [ ]:
# Логарифмирование улучшило распределение. Создадим столбец с логарифмом.
df['model_old_log'] = np.log(df['model_old']+1)

### Владельцы

In [ ]:
df.Владельцы.unique()

In [ ]:
# Поставим примерное количество владельцев в зависимости от возраста машины 
df.Владельцы = df.Владельцы.apply(set_None)
df.Владельцы = df.apply(lambda row: row['Владельцы'] if not(row['Владельцы'] is None) \
                                  else '1' if row['modelDate']>2014 \
                                  else '2' if row['modelDate']>2007 \
                                  else '3' , axis=1)

In [ ]:
# Приведем к целым значениям
df['Владельцы'] = df['Владельцы'].apply(lambda x: 3 if '3' in x else 2 if '2' in x else 1 if '1' in x else 0)

In [ ]:
df.Владельцы.unique()

### Владение

In [ ]:
print(df.Владение.unique())

In [ ]:
# Функция преобразования текстового значения в месяцы
def get_months(x):
    if type(x) != list:
        return x
    elif len(x) == 5:
        return int(x[0])*12 + int(x[3]) 
    elif len(x) == 2 and 'месяц' in x[1]:
        return int(x[0]) 
    else:
        return int(x[0])*12
    
# Получим список составных частей срока владения
df['Владение'] = df['Владение'].apply(set_None)
df['Владение'] = df['Владение'].apply(lambda x: '0 лет и 0 месяцев' if pd.isna(x) else x)
df['own_num'] = df['Владение'].apply(lambda x: x.split())
display(df['own_num'])

# Превратим этот список в количество месяцев
df['own_num'] = df['own_num'].apply(get_months)
display(df['own_num'])

In [ ]:
# Заполним пропуски значением в зависмости от возраста. Считаем, что в среднем владеют 7 лет=84 месяцев.
# Если возраст машины менее 7 лет, то напишем возраст машины. Иначе поставим 7 лет.
# Можно также пойти от возраста автомобиля и количества его владельцев - поделить возраст на владельцев. Непонятно, что лучше.
df.own_num = df.apply(lambda row: row['own_num'] if not pd.isna(row['own_num']) \
                                  else row['age']*12 if row['age']<7 \
                                  else 84, axis=1)
df.own_num = np.round(df.own_num/12,2)

In [ ]:
print(df.own_num.unique())

In [ ]:
# Посмотрим на влияние логарифма 
val_log_plot(df, 'own_num')
val_log_plot(df[df['sample']==1], 'own_num')
val_log_plot(df[df['sample']==0], 'own_num')

Если не считать новые автомобили, которыми никто не владел, то остальные распределены лучше в логарифмическом виде.

In [ ]:
# Логарифмируем
df['own_num_log'] = np.log(df['own_num']+1)

### name

In [ ]:
print(df.name.unique())

In [ ]:
df.name = df.name.apply(set_lower)

In [ ]:
print(len(df[df.name.str.contains('drive')]))
print(len(df[df.name.str.contains('blue')]))
print(len(df[df.name.str.contains('competition')]))
print(len(df[df.name.str.contains('hyb')]))
print(len(df[(df.name.str.contains('long')) | (df.name.str.contains('длинн'))]))
print(len(df[(df.name.str.contains('compact')) | (df.name.str.contains('компакт'))]))
print(len(df[df.name.str.contains('tronic')]))
print(len(df[df.name.str.contains('speed')]))
print(len(df[df.name.str.contains('tfsi')]))

Наделаем новых признаков, которые встречаются у большого количества автомобилей.

In [ ]:
df['drive'] = df['name'].apply(lambda x: 1 if 'drive' in x else 0)
df['blue'] = df['name'].apply(lambda x: 1 if ('blue' in x or 'clean' in x) else 0)
df['long'] = df['name'].apply(lambda x: 1 if ('long' in x or 'длин' in x) else 0)

### description

In [ ]:
# Сделаем первичную обработку
df['description'] = df['description'].apply(set_None)
df['description'] = df['description'].apply(set_lower)

# Заменим None на пустую строку
df['description'] = df['description'].apply(lambda x: '' if x is None else x)

# Заменим перевод строки '\n' и лишние знаки на ' '
intab = '.,;:!?-•–/+"☑️✔➥●☛✅ё' 
outtab = '                   е' 
trantab = str.maketrans(intab, outtab)
df['description'] = df['description'].apply(lambda x: x.replace('\n', ' '))
df['description'] = df['description'].apply(lambda x: x.translate(trantab))
df['description'] = df['description'].apply(lambda x: x.replace('\uf043\t', ' '))
df['description'] = df['description'].apply(lambda x: x.replace('\uf0fe\t', ' ').replace('  ', ' '))
df['description'] = df['description'].apply(lambda x: re.sub('\w*\d\w*', '', x))

In [ ]:
df['description'][4668]

In [ ]:
# Выясним тональность отзыва и добавим в качестве фичи
from textblob import TextBlob
df['desc_polarity'] = df['description'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['desc_polarity'].value_counts()

In [ ]:
df['desc_subjectivity'] = df['description'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
df['desc_subjectivity'].value_counts()

In [ ]:
# Добавим столбец с длиной описания
df['desc_len'] = df.description.apply(lambda x: 0 if pd.isna(x) else len(x))

In [ ]:
# Построим разреженные матрицы слов и создадим новые признаки: 
#   среднее значение (наполненность матрицы) и количество существенных слов
stopwords=nltk.corpus.stopwords.words('russian')
newStopWords = ['автомобиль','автомобилей', 'автомобиля', 'продаю', 'пробег', 'машина']
stopwords.extend(newStopWords)

vectorizer = CountVectorizer(stop_words=stopwords)

text_feat = vectorizer.fit_transform(df['description'])
df['desc_mean'] = text_feat.mean(axis=1)*1000
df['desc_sum'] = text_feat.sum(axis=1)

In [ ]:
# Посмотрим на влияние логарифма на количество слов в описании
val_log_plot(df, 'desc_mean')
val_log_plot(df[df['sample']==1], 'desc_mean')
val_log_plot(df[df['sample']==0], 'desc_mean')

val_log_plot(df, 'desc_sum')
val_log_plot(df[df['sample']==1], 'desc_sum')
val_log_plot(df[df['sample']==0], 'desc_sum')

val_log_plot(df, 'desc_len')
val_log_plot(df[df['sample']==1], 'desc_len')
val_log_plot(df[df['sample']==0], 'desc_len')

Распределение всех величин при логарифмировании становится более похожим на нормальное. Заменим все величины логарифмом.

In [ ]:
df['desc_mean_log'] = np.log(df['desc_mean'] + 1)
df['desc_sum_log'] = np.log(df['desc_sum'] + 1)
df['desc_len_log'] = np.log(df['desc_len'] + 1)

In [ ]:
# Разбираем на слова признак 'description'
description = df['description'].copy(deep=True)
tokenizer = CountVectorizer(stop_words=stopwords).build_analyzer()
tokenized_text_feature = description.apply(tokenizer)

# Создаем разреженную матрицу слов
tf_idf = TfidfVectorizer(max_features=50, stop_words=stopwords)
tf_idf_feature = tf_idf.fit_transform(description).toarray()

# Уменьшаем до 2 размерность матрицы слов
pca = PCA(n_components=2, random_state=0)
tf_idf_pc = pca.fit_transform(tf_idf_feature)
df_tfidf = pd.DataFrame(tf_idf_pc, columns=['tfidf1', 'tfidf2'])

# Добавляем к данным матрицу слов со сниженной размерностью
df = pd.concat([df, df_tfidf], axis=1)
df[['tfidf1', 'tfidf2']]

In [ ]:
# Извлекаем дополнительные признаки
df['exState'] = df.description.apply(lambda x: 1 if re.search(r'отличн.+состоян+',x) else 0)
df['goodState'] = df.description.apply(lambda x: 1 if re.search(r'хорош.+состоян+',x) else 0)
df['noSmoke'] = df.description.apply(lambda x: 1 if 'прокур' in x or 'курил' in x else 0)
df['dent']= df.description.apply(lambda x: 1 if 'вмят' in x or 'царап' in x or 'трещ' in x or 'тресн' in x else 0)
df['salon']= df.description.apply(lambda x: 1 if 'дилер' in x or 'ликвидац' in x or 'кредит' in x \
                                      or 'юридич' in x or 'traide' in x or 'трейд' in x or 'официал' in x else 0)
df['carter'] = df.description.apply(lambda x: 1 if re.search(r'защит.+картер+',x) else 0)

df['electro-window'] = df.description.apply(lambda x: 1 if 'электростеклоподъемник' in x else 0)
df['airbag'] = df.description.apply(lambda x: 1 if re.search(r'подушк', x) else 0)
df['wheel-power'] = df.description.apply(lambda x: 1 if re.search(r'усилител', x) and re.search(r'руля', x) else 0)
df['lock'] = df.description.apply(lambda x: 1 if re.search(r'центральн.+зам+',x) else 0)
df['help'] = df.description.apply(lambda x: 1 if re.search(r'систем.+помощ+',x) else 0)
df['climate'] = df.description.apply(lambda x: 1 if 'климат' in x else 0)
df['cruise'] = df.description.apply(lambda x: 1 if 'круиз' in x else 0)
df['computer'] = df.description.apply(lambda x: 1 if re.search(r'бортов.+компьютер+',x) else 0)
df['heat'] = df.description.apply(lambda x: 1 if 'подогрев' in x else 0)
df['electro-mirrors'] = df.description.apply(lambda x: 1 if 'электропривод зеркал' in x else 0)
df['ptf'] = df.description.apply(lambda x: 1 if 'противотуман' in x else 0)
df['abs'] = df.description.apply(lambda x: 1 if 'антиблокировочн' in x or 'abs' in x or 'абс' in x else 0)
df['esp'] = df.description.apply(lambda x: 1 if 'курсовой устойчивости' in x or 'esp' in x else 0)
df['condition'] = df.description.apply(lambda x: 1 if 'кондиционер' in x else 0)
df['immo'] = df.description.apply(lambda x: 1 if 'иммобил' in x else 0)
df['alarm'] = df.description.apply(lambda x: 1 if 'сигнализ' in x else 0)
df['navigation'] = df.description.apply(lambda x: 1 if 'навига' in x else 0)
df['park'] = df.description.apply(lambda x: 1 if 'парк' in x else 0)
df['audio'] = df.description.apply(lambda x: 1 if 'audio' in x or 'аудио' in x else 0)
df['rain-sensor'] = df.description.apply(lambda x: 1 if re.search(r'датчик.+дождя+',x) else 0)
df['alloy-wheel-disks'] = df.description.apply(lambda x: 1 if re.search(r'легкосплавн',x) else 0)
df['camera'] = df.description.apply(lambda x: 1 if 'камера' in x else 0)
df['tyre-pressure'] = df.description.apply(lambda x: 1 if 'в шинах' in x else 0)
df['halogen'] = df.description.apply(lambda x: 1 if 'галоген' in x else 0)
df['dark'] = df.description.apply(lambda x: 1 if re.search(r'темн.+салон+',x) or re.search(r'черн.+салон+',x) else 0)
df['asr'] = df.description.apply(lambda x: 1 if 'антипробуксов' in x or 'asr' in x else 0)

# Анализ значимости признаков

In [ ]:
df.columns

In [ ]:
# Двоичные признаки 
bin_columns = ['ПТС', 'Руль', 'color_type',
       'eP_log_out', 'eD_log_out', 'mileage_out',
       'drive', 'blue', 'long', 
       'exState', 'goodState', 'noSmoke', 'dent', 'salon', 'carter',
       'electro-window', 'airbag', 'wheel-power', 'lock', 'help', 'climate',
       'cruise', 'computer', 'heat', 'electro-mirrors', 'ptf', 'abs', 'esp',
       'condition', 'immo', 'alarm', 'navigation', 'park', 'audio',
       'rain-sensor', 'alloy-wheel-disks', 'camera', 'tyre-pressure',
       'halogen', 'dark', 'asr']

# Числовые признаки
num_columns = ['Владельцы','tax_rate','eP_log', 'eD_log', 
       'mileage_norm_log', 'milPerYear', 'model_age_log', 'age_log', 'model_old_log',
       'own_num_log', 'desc_polarity', 'desc_subjectivity', 
       'desc_mean_log', 'desc_sum_log', 'desc_len_log', 'tfidf1', 'tfidf2']

# Категориальные признаки
cat_columns = ['bodyType', 'brand', 'color', 'fuelType', 'model_info', 
       'numberOfDoors', 'vehicleTransmission', 'body']

# Служебные признаки
srv_columns = ['price', 'price_log', 'sample', 'sell_id']

In [ ]:
# Проверку значимости будем делать на копии подготовленного датасета
data = df.copy()

In [ ]:
# Собираем столбцы и кодируем категориальные переменные.
# Для нейронной сети будем применять One-Hot-Coding.
# Здесь, для проверки значимости, воспользуемся более простым кодером.
columns = bin_columns + num_columns + cat_columns +srv_columns
for colum in cat_columns:
    data[colum] = data[colum].astype('category').cat.codes

In [ ]:
# Проверим корреляцию числовых столбцов в сборном датасете
correlation = data[num_columns+['price']].corr()
plt.figure(figsize=(16, 12))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

Столбцы desc_mean_log, desc_len_log, desc_sum_log сильно коррелируют. Уберем из признаков для модели desc_sum_log. Все остальные столбцы оставим, даже с корреляцией 0.95. Опыт показывает, что удаление ниже такого коэффициента уменьшает метрику.

In [ ]:
# Проверим статистическую значимость числовых столбцов по Пирсону
get_stat_corr(data[data.price>0], num_columns, 'price_log')

Все числовые столбцы значимы для цены.

In [ ]:
# Посмотрим на значимость категориальных столбцов
df1 = data[data.price>0][columns].copy()

label_encoder = LabelEncoder()
for col in cat_columns:
    df1[col] = label_encoder.fit_transform(df1[col])
    
imp_cat = pd.Series(mutual_info_classif(df1[cat_columns], 
                                        df1['price'], discrete_features = True), 
                    index=cat_columns)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

plt.show()

Все категориальные столбцы тоже значимы.

In [ ]:
# Проверим значимость признаков с дискретными значениями по критерию Стьюдента
get_stat_dif(df1, cat_columns, 'price_log')

Этот тест показал, что цвет не влияет на цену. Но так как предыдущий тест дал этому признаку второе место по значимости, все-таки оставим.

In [ ]:
# Проверим значимость двоичных признаков
imp_cat = pd.Series(mutual_info_classif(df1[bin_columns], 
                                        df1['price'], discrete_features = True), 
                    index=bin_columns)
imp_cat.sort_values(inplace = True)
plt.figure(figsize=(8, 8)) 
imp_cat.plot(kind = 'barh')
plt.show()

Исключим последние по значимости признаки (меньше 0.05). Это Руль, eP_log_out, halogen, mileage_out, blue.

In [ ]:
# Собираем столбцы, которые показали значимость

# Двоичные признаки 
bin_columns = ['ПТС', 'color_type','eD_log_out', 'drive', 'long', 
       'exState', 'goodState', 'noSmoke', 'dent', 'salon', 'carter',
       'electro-window', 'airbag', 'wheel-power', 'lock', 'help', 'climate',
       'cruise', 'computer', 'heat', 'electro-mirrors', 'ptf', 'abs', 'esp',
       'condition', 'immo', 'alarm', 'navigation', 'park', 'audio',
       'rain-sensor', 'alloy-wheel-disks', 'camera', 'tyre-pressure',
       'dark', 'asr']

# Числовые признаки
num_columns = ['Владельцы','tax_rate','eP_log', 'eD_log', 
       'mileage_norm_log', 'milPerYear', 'model_age_log', 'age_log', 'model_old_log',
       'own_num_log', 'desc_polarity', 'desc_subjectivity', 
       'desc_mean_log', 'desc_len_log', 'tfidf1', 'tfidf2']

# Категориальные признаки
cat_columns = ['bodyType', 'brand', 'color', 'fuelType', 'model_info', 
       'numberOfDoors', 'vehicleTransmission', 'body']

# Служебные признаки
srv_columns = ['price', 'sample', 'sell_id']

# Добавление столбцов со статистикой

Добавим столбцы со статистиками по важным категориям (бренд, модель, кузов, привод, трансмиссия, топливо) + 
и важным двоичным признакам для самых важных числовых признаков.    
Важность признаков определили по результатам корреляционного анализа.    
    
Статистику возьмем только по автомобилям с пробегом

In [ ]:
df0 = df[df.mileage>0]

num_feature = ['price', 'eP_log', 'eD_log', 
        'mileage_norm_log', 'milPerYear', 'model_age_log', 'age_log']
cat_feature = ['model_info','bodyType', 'brand', 'color', 'fuelType',  
       'numberOfDoors', 'vehicleTransmission', 'body'] 

In [ ]:
new_columns = []

for nf in num_feature:
    for cf in cat_feature:
        if nf=='price':
            mean_nf = df0[df0['sample']==1][nf].mean()
            median_nf = df0[df0['sample']==1][nf].median()
            max_nf = df0[df0['sample']==1][nf].max()
            min_nf = df0[df0['sample']==1][nf].min()
            std_nf = df0[df0['sample']==1][nf].std()
        else:
            mean_nf = df0[nf].mean()
            median_nf = df0[nf].median()
            max_nf = df0[nf].max()
            min_nf = df0[nf].min()
            std_nf = df0[nf].std()
        
        # Среднее
        match = dict(df.groupby(cf)[nf].mean())
        df['mean_'+cf+'_'+nf] = np.log(df[cf].apply(lambda x: match[x] if x in match else mean_nf )+1)
        df['mean_'+cf+'_'+nf] = df['mean_'+cf+'_'+nf].fillna(mean_nf)
        new_columns.append('mean_'+cf+'_'+nf)
        
        # Медиана
        match = dict(df.groupby(cf)[nf].median())
        df['median_'+cf+'_'+nf] = np.log(df[cf].apply(lambda x: match[x] if x in match else median_nf )+1)
        df['median_'+cf+'_'+nf] = df['median_'+cf+'_'+nf].fillna(median_nf)
        new_columns.append('median_'+cf+'_'+nf)
                
        # Максимум
        match = dict(df.groupby(cf)[nf].max())
        df['max_'+cf+'_'+nf] = np.log(df[cf].apply(lambda x: match[x] if x in match else max_nf )+1)
        df['max_'+cf+'_'+nf] = df['max_'+cf+'_'+nf].fillna(max_nf)
        new_columns.append('max_'+cf+'_'+nf)
        
        # Минимум
        match = dict(df.groupby(cf)[nf].min())
        df['min_'+cf+'_'+nf] = np.log(df[cf].apply(lambda x: match[x] if x in match else min_nf )+1)
        df['min_'+cf+'_'+nf] = df['min_'+cf+'_'+nf].fillna(min_nf)
        new_columns.append('min_'+cf+'_'+nf)
        
        # Разброс
        match = dict(df.groupby(cf)[nf].std())
        df['std_'+cf+'_'+nf] = np.log(df[cf].apply(lambda x: match[x] if x in match else std_nf )+1)
        df['std_'+cf+'_'+nf] = df['std_'+cf+'_'+nf].fillna(std_nf)
        new_columns.append('std_'+cf+'_'+nf)

In [ ]:
columns = bin_columns + num_columns + cat_columns + srv_columns + new_columns
columns1 = bin_columns + num_columns + cat_columns + srv_columns
columns2 = srv_columns + new_columns

In [ ]:
df.info()

# Сохранение

Сохраним обработанный дадасет и список значимых столбцов.

In [ ]:
df[columns1].to_csv('df_clean1.csv', index=False)
df[columns2].to_csv('df_clean2.csv', index=False)

In [ ]:
# columns = bin_columns + num_columns + cat_columns + srv_columns

In [ ]:
with open('best_columns', 'w') as f:
    f.write("\n".join(columns))

In [ ]:
# Пример чтения
# with open('./best_columns', 'r') as f:
#     mystring = f.read()
# my_list = mystring.split("\n")

# Подбор моделей

### Подготовка данных

In [ ]:
data = df[columns].copy()
data = pd.get_dummies(data, columns=cat_columns, dummy_na=False)

In [ ]:
X = data[data['sample'] == 1].drop(['sample','price','sell_id'], axis=1)
X_sub = data[data['sample'] == 0].drop(['sample','price','sell_id'], axis=1)
y = data[data['sample'] == 1]['price']
X.shape, len(y)

In [ ]:
# Разбиваем тренировочный набор для обучения
VAL_SIZE = 0.2
RANDOM_SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

### RandomForest

In [ ]:
# Модель с параметрами по умолчанию на старых данных
model1 = RandomForestRegressor(random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
model1.fit(X_train, np.log(y_train))
y_pred1 = model1.predict(X_test)

# Преобразуем y_test, y_pred к exp значениям для оценки MAPE

y_pred1 = np.round(np.exp(y_pred1))

# Вывод результата MAPE
print(f"Точность обученной модели по метрике MAPE: {(mape(y_test, y_pred1))*100:0.2f}%")

# CatBoost

In [ ]:
model2 = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model2.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

# Непонятно, зачем это. Надо почитать.
# model2.save_model('catboost_single_model_2_baseline.model')

y_pred2 = np.exp(model2.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred2))*100:0.2f}%") 

In [ ]:
# Модель с параметрами по умолчанию 
model3 = GradientBoostingRegressor(random_state=RANDOM_SEED) 

# Обучаем модель на тестовом наборе данных
model3.fit(X_train, np.log(y_train))
y_pred3 = model3.predict(X_test)

# Преобразуем y_test, y_pred к exp значениям для оценки MAPE 

y_pred3 = np.round(np.exp(y_pred3))

# Вывод результата MAPE 
print(f"Точность обученной модели по метрике MAPE: {(mape(y_test, y_pred3))*100:0.2f}%")

In [ ]:
# Модель с параметрами по умолчанию
model4 = xgb.XGBRegressor()
model4.fit(X_train, np.log(y_train))

y_pred4 = np.exp(model4.predict(X_test))

# Вывод результата MAPE
print(f"Точность обученной модели по метрике MAPE: {(mape(y_test, y_pred4))*100:0.2f}%")

In [ ]:
y_pred = 0.33*y_pred1 + 0.67*y_pred2
print(f"Точность обученной модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

# Submission

In [ ]:
predict_submission1 = np.exp(model1.predict(X_sub))
predict_submission2 = np.exp(model2.predict(X_sub))
predict_submission = (predict_submission1 + predict_submission2)/2

sample_submission['price'] = predict_submission
sample_submission.to_csv('submission_ml.csv', index=False)
sample_submission.head(10)